#  Mission to Mars ETL Project

## Import Required Modules 

In [186]:
#Import our Modules
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import time 

## ETL Local Variables 

In [360]:
site_base = 'https://astrogeology.usgs.gov'
mars_latest_news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
mars_gallery_url = 'https://www.jpl.nasa.gov/images?search='
mars_facts_url = 'https://space-facts.com/mars/'
mars_hemisphers_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced'

## Get Latest News

In [235]:
html = requests.get(mars_latest_news_url)

bs = BeautifulSoup(html.text,'html.parser')



In [239]:
results = bs.find("div",class_="slide")
print(results)
paragraph = results.a.text.strip()

title = results.find('div',class_="content_title").text.strip()


<div class="slide">
<div class="image_and_description_container">
<a href="/news/8896/nasa-ingenuity-mars-helicopter-prepares-for-first-flight/">
<div class="rollover_description">
<div class="rollover_description_inner">
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  
</div>
<div class="overlay_arrow">
<img alt="More" src="/assets/overlay-arrow.png"/>
</div>
</div>
<img alt="NASA Ingenuity Mars Helicopter Prepares for First Flight" class="img-lazy" data-lazy="/system/news_items/list_view_images/8896_thumb-320.jpg" src="/assets/loading_320x240.png"/>
</a>
</div>
<div class="content_title">
<a href="/news/8896/nasa-ingenuity-mars-helicopter-prepares-for-first-flight/">
NASA Ingenuity Mars Helicopter Prepares for First Flight
</a>
</div>
</div>


In [240]:
print(title)
print(paragraph)

NASA Ingenuity Mars Helicopter Prepares for First Flight
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.


In [127]:
title = title_str.text

In [128]:
print(title)


NASA's Perseverance Drives on Mars' Terrain for First Time



## Splinter Work

In [334]:
# Setup splinter browser driver

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(mars_gallery_url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\mreg9\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [311]:


browser.find_by_css("input[id=filter_Mars]").first.click()
    
bs = BeautifulSoup(browser.html,'html.parser')
result = bs.find("div", class_='SearchResultCard')

In [241]:
bs = BeautifulSoup(browser.html,'html.parser')
result = bs.find("div", class_='SearchResultCard')

In [242]:
print(result)

<div class="SearchResultCard" page_id="127955" page_score="1"><a class="group cursor-pointer block" href="/images/gale-crater-false-color-11"><div class="border-gray-light-mid lg:pb-10 lg:mb-10 relative grid grid-cols-9 gap-5 pb-6 mb-6 border-b"><div class="lg:hidden col-span-full text-subtitle"><span>
        Image
        <span class="sr-only">.</span></span> <!-- --> <!-- --></div> <div class="sm:col-span-6 md:col-span-6 lg:col-span-7 xl:col-span-7 col-span-6"><div class="text-subtitle lg:block hidden mb-3"><span>
          Image
          <span class="sr-only">.</span></span> <!-- --> <!-- --></div> <h2 class="mb-3 text-h5">Gale Crater - False Color</h2> <!-- --> <p class="text-gray-mid-dark">Mar 25, 2021</p></div> <div class="SearchResultCard-image xl:pl-14 sm:col-span-3 md:col-span-3 lg:col-span-2 xl:col-span-2 col-span-3"><div data-v-a6031820=""><div class="BaseImagePlaceholder aspect-ratio-one-one dark-theme" data-v-a6031820=""><div class="sm:object-cover object-cover" data-v-a

In [247]:
large_image_html = result.a['href']

In [249]:
browser.click_link_by_href(large_image_html)

D:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:474: FutureWarning: browser.find_link_by_href is deprecated. Use browser.links.find_by_href instead.
  warnings.warn(


In [255]:
featured_image_url = result.img['src']
print(featured_image_url)

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24509.2e16d0ba.fill-400x400-c50.jpg


##  Mars Facts Table Scraping With Pandas

In [276]:
html = requests.get(mars_facts_url)

In [277]:
df = pd.read_html(html.text)
mars_facts_df = df[0]

cols = ["Stats", "Values"]

mars_facts_df.set_axis(cols,axis="columns",inplace=True)
mars_facts_df.set_index("Stats",inplace=True)

In [278]:
mars_facts_df

,Values
Stats,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [279]:
mars_facts_html_table = mars_facts_df.to_html()

In [280]:
mars_facts_html_table = mars_facts_html_table.replace('\n', '')

In [351]:
browser.visit(mars_hemisphers_url)

In [352]:
bs = BeautifulSoup(browser.html,'html.parser')

In [353]:
results = bs.find_all('div', class_="item")

In [364]:
hemi_image_url_list = []
for result in results:
    hemi_dict = {}
    link = result.a['href'].strip()
    print(link)
    title = result.h3.text
    print(title)
    browser.visit(site_base + link)
    #browser.click_link_by_href(link)
    bs = BeautifulSoup(browser.html,'html.parser')
    res = bs.find('img',class_='wide-image')
    img_url = site_base + res['src']
    hemi_dict = {"title":title, "img_url":img_url}
    hemi_image_url_list.append(hemi_dict)
    #browser.back()
#for result in results:
#    print(result['href'].text)

/search/map/Mars/Viking/cerberus_enhanced
Cerberus Hemisphere Enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
Schiaparelli Hemisphere Enhanced
/search/map/Mars/Viking/syrtis_major_enhanced
Syrtis Major Hemisphere Enhanced
/search/map/Mars/Viking/valles_marineris_enhanced
Valles Marineris Hemisphere Enhanced


In [365]:
print(hemi_image_url_list)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
